In [5]:
import os
import sys
import re
import json
from datasets import (load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict,
    Value,
    Features
)

In [6]:
import torch
import random
import pandas as pd
import numpy as np
import collections
import matplotlib.pyplot as plt

In [7]:
from transformers import AutoTokenizer
from tqdm.notebook import tqdm

## Load Dataset

In [8]:
paper_dataset = load_dataset('metamong1/summarization_paper', 
    use_auth_token='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')

Reusing dataset paper_summarization (/opt/ml/.cache/huggingface/datasets/metamong1___paper_summarization/Paper Summarization/2.2.0/46d835d4e22daa3a5a46d13de39e3d75f6c2eaef5ead153d48cbe8d7cd3bec9c)


  0%|          | 0/2 [00:00<?, ?it/s]

In [12]:
sys.path.append('../')

In [13]:
from preprocessor import PaperPreprocessor

In [14]:
preprocessor = PaperPreprocessor()

In [15]:
paper_dataset = paper_dataset.map(preprocessor.for_train)

  0%|          | 0/73640 [00:00<?, ?ex/s]

  0%|          | 0/18411 [00:00<?, ?ex/s]

In [16]:
train_data = paper_dataset['train']
train_docs = [data['text'] for data in train_data]

In [17]:
train_docs[0]

'역시간 구조보정은 음원영역 파동장 외삽과 수진기영역 파동장 외삽의 상호상관으로 지층구조를 영상화하는 방법으로 복잡한 등방성 매질 층서구조를 영상화하는데 주로 이용된다. 그러나 일반적으로 지구내부 지층구조는 이방성 특성을 지니고 있으므로 이방성을 고려한 구조보정 기술이 필요하다. 여기에서는 편미분 파동장과 음원모음의 내적에 의한 알고리즘과 가상음원과 역전파 파동장과의 내적에 의한 알고리즘을 이용하여 횡적등방성 매질에서 역시간 구조보정 기술을 개발하고자 하였다. 단순 이방성 지층모델에 대한 수치모형실험 결과, 두 가지 방법에 의한 지층단면도 영상은 거의 차이가 없어 가상음원과 역전파 파동장과의 내적으로 구조보정을 실시하는 것이 효과적임을 알 수 있었다. 수평적으로 속도가 변하는 이방성 매질 지층구조에서 편미분 파동장을 구하지 않고 영상화 할 수 있음을 알 수 있었다.'

## Select UNK Token

In [26]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large')

In [29]:
from nltk.tokenize import sent_tokenize, word_tokenize

In [31]:
sent_tokenize(train_docs[0])

['역시간 구조보정은 음원영역 파동장 외삽과 수진기영역 파동장 외삽의 상호상관으로 지층구조를 영상화하는 방법으로 복잡한 등방성 매질 층서구조를 영상화하는데 주로 이용된다.',
 '그러나 일반적으로 지구내부 지층구조는 이방성 특성을 지니고 있으므로 이방성을 고려한 구조보정 기술이 필요하다.',
 '여기에서는 편미분 파동장과 음원모음의 내적에 의한 알고리즘과 가상음원과 역전파 파동장과의 내적에 의한 알고리즘을 이용하여 횡적등방성 매질에서 역시간 구조보정 기술을 개발하고자 하였다.',
 '단순 이방성 지층모델에 대한 수치모형실험 결과, 두 가지 방법에 의한 지층단면도 영상은 거의 차이가 없어 가상음원과 역전파 파동장과의 내적으로 구조보정을 실시하는 것이 효과적임을 알 수 있었다.',
 '수평적으로 속도가 변하는 이방성 매질 지층구조에서 편미분 파동장을 구하지 않고 영상화 할 수 있음을 알 수 있었다.']

In [39]:
unk_words = []

In [40]:
for doc in tqdm(train_docs) :
    sen_list = sent_tokenize(doc)

    for sen in sen_list :
        word_list = word_tokenize(sen)

        for word in word_list :
            if tokenizer.unk_token_id in tokenizer.encode(word) :
                unk_words.append(word)

  0%|          | 0/73640 [00:00<?, ?it/s]

In [41]:
unk_word_counter = collections.Counter()
unk_word_counter.update(unk_words)

unk_word_counter = dict(unk_word_counter)
print('Unk Word Size : %d' %len(unk_word_counter))

Unk Word Size : 26921


In [42]:
unk_word_item = sorted(unk_word_counter.items(), key=lambda x : x[1], reverse=True)

In [43]:
unk_word_item[:10]

[('휨', 141),
 ('살핌으로써', 53),
 ('앎과', 45),
 ('앎', 41),
 ('앎의', 37),
 ('휨강도', 37),
 ('시퀀스를', 34),
 ('앎을', 33),
 ('시퀀스', 32),
 ('朱子의', 30)]

In [44]:
words = [item[0] for item in unk_word_item]
counts = [item[1] for item in unk_word_item]

In [47]:
unk_word_df = pd.DataFrame({'Word' : words, 'Counts' : counts})
unk_word_df.to_csv('../paper_vocabs.csv')

In [48]:
unk_word_df.head()

,Word,Counts
0,휨,141
1,살핌으로써,53
2,앎과,45
3,앎,41
4,앎의,37


## Tokenizer

In [65]:
tokenizer = AutoTokenizer.from_pretrained('klue/roberta-large', use_fast=True)

In [66]:
print('Index : 31500 \t Token : %s' %tokenizer.convert_ids_to_tokens(31500))
print('Index : 31599 \t Token : %s' %tokenizer.convert_ids_to_tokens(31599))

Index : 31500 	 Token : [unused0]
Index : 31599 	 Token : [unused99]


In [67]:
tokenizer.save_pretrained('../Tokenizer')

('../Tokenizer/tokenizer_config.json',
 '../Tokenizer/special_tokens_map.json',
 '../Tokenizer/vocab.txt',
 '../Tokenizer/added_tokens.json',
 '../Tokenizer/tokenizer.json')

In [68]:
from transformers import PreTrainedTokenizerFast

In [69]:
words[:5]

['휨', '살핌으로써', '앎과', '앎', '앎의']

In [70]:
class TokenizerOptimization :
    def __init__(self, dir_path, word_list) :
        self.txt_path = os.path.join(dir_path, 'vocab.txt')
        self.json_path = os.path.join(dir_path, 'tokenizer.json')
        self.vocab_map = {i: word for i, word in enumerate(word_list)}

    def write_vocab(self, vocab_map) :
        data_size = len(vocab_map)
        vocab_list = list(vocab_map.values())
        f = open(self.txt_path, 'w')
        for i in range(data_size):
            f.write(vocab_list[i]+'\n')
        f.close()

    def add_unused(self, vocab_map, tokenizer) :
        unused_start = tokenizer.convert_tokens_to_ids('[unused0]')
        unused_end = tokenizer.convert_tokens_to_ids('[unused99]') + 1

        unused_size = unused_end - unused_start 
        for i in range(unused_size) :
            unused_idx = unused_start + i
            word = vocab_map[i]
            vocab_map[unused_idx] = word

    def load_tokenizer_json(self) :
        with open(self.json_path, "r") as json_data:
            tokenizer_data = json.load(json_data)
        return tokenizer_data

    def write_tokenizer_json(self, tokenizer_data, vocab_data) :
        inverse_vocab_data = {vocab_data[key] : key for key in vocab_data.keys()}
        tokenizer_data['model']['vocab'] = inverse_vocab_data
        with open(self.json_path, 'w') as json_file:
            json.dump(tokenizer_data, json_file)

    def optimize(self, tokenizer) :
        assert isinstance(tokenizer, PreTrainedTokenizerFast)
        self.add_unused(self.vocab_map, tokenizer)
        self.write_vocab(self.vocab_map)

        tokenizer_data = self.load_tokenizer_json()
        self.write_tokenizer_json(tokenizer_data, self.vocab_map)

In [71]:
optimizer = TokenizerOptimization('./', words)

In [72]:
optimizer.optimize(tokenizer)

In [73]:
new_tokenizer = AutoTokenizer.from_pretrained('../Tokenizer', use_fast=True)

In [74]:
print('Index : 31500 \t Token : %s' %new_tokenizer.convert_ids_to_tokens(31500))
print('Index : 31599 \t Token : %s' %new_tokenizer.convert_ids_to_tokens(31599))

Index : 31500 	 Token : 휨
Index : 31599 	 Token : 茶
